<a href="https://colab.research.google.com/github/itissandeep98/StackOverFlow-Tag-Predictor/blob/master/MLproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import pandas as pd 
import csv,nltk,random,pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer,SnowballStemmer
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,ShuffleSplit
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
import pickle
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Offline Preprocessing

In [ ]:
af=pd.read_csv('/content/drive/MyDrive/Ml Project/Train.zip')
df=af[:1000000]

duplicate_pairs = df.sort_values('Title', ascending=False).duplicated('Title')
df = df[~duplicate_pairs]

df["Tags"]=df["Tags"].astype(str)
df["tag_count"] = df["Tags"].apply(lambda x : len(x.split()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [ ]:
# Sandeep Dont Delete This 
df.head()

In [ ]:
qus_list=[]

for index,row in df.iterrows():
    title, body, tags = row["Title"], row["Body"], row["Tags"]   
    body=re.sub('<code>(.*?)</code>', '', body, flags=re.MULTILINE|re.DOTALL)
    body = re.sub('<.*?>', ' ', str(body.encode('utf-8')))
    title=title.encode('utf-8')
    question=str(title)+" "+str(body)
    question=re.sub(r'[^A-Za-z]+',' ',question)
    words=word_tokenize(str(question.lower()))
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer("english")
    question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    qus_list.append(question)
   
df["question"] = qus_list

In [ ]:
# Sandeep Dont delete This
preprocessed_df = df[["question","Tags"]]
preprocessed_df

NameError: ignored

In [ ]:
preprocessed_df.to_csv("preprocessed_20k.csv")

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Ml Project/preprocessed_file.csv')
preprocessed_df = df[["question","Tags"]]

In [ ]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
multilabel_y = vectorizer.fit_transform(preprocessed_df['Tags'])

In [ ]:
def tags_to_consider(n):
    tag_i_sum = multilabel_y.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=multilabel_y[:,sorted_tags_i[:n]]
    #names=[vectorizer.get_feature_names() for i in multilabel_y[:,sorted_tags_i[:n]] ]
    return yn_multilabel

def questions_explained_fn(n):
    multilabel_yn = tags_to_consider(n)
    x= multilabel_yn.sum(axis=1)
    return (np.count_nonzero(x==0))

In [ ]:
questions_explained = []
total_tags=multilabel_y.shape[1]
total_qs=preprocessed_df.shape[0]
for i in range(500, total_tags, 100):
    questions_explained.append(np.round(((total_qs-questions_explained_fn(i))/total_qs)*100,3))


In [ ]:
yx_multilabel = tags_to_consider(5500)
#print("number of questions that are not covered :", questions_explained_fn(5500),"out of ", total_qs)
#print("Questions covered ", (total_qs -questions_explained_fn(5500) )*100 / total_qs )

In [ ]:
names=vectorizer.get_feature_names()
tag_i_sum = multilabel_y.sum(axis=0).tolist()[0]
sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
ans=[]
for i in sorted_tags_i:
    ans.append(names[i])
np.savetxt("tag_names.csv",ans, delimiter =", ",  fmt ='% s') 
print(ans)

# Spliting Data into training and testing


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Ml Project/preprocessed_20k.csv')
preprocessed_df = df[["question","Tags"]]
df

,Unnamed: 0,question,Tags
0,0,check upload file imag without mime type like ...,php image-processing file-upload upload mime-t...
1,1,prevent firefox close press ctrl favorit edito...,firefox
2,2,error invalid type list variabl import matlab ...,r matlab machine-learning
3,3,replac special charact url probabl simpl simpl...,c# url encoding
4,4,modifi whoi contact detail use modifi function...,php api file-get-contents
...,...,...,...
19976,19995,instal interbas firebird conflict interbas fir...,firebird interbase
19977,19996,chang text color activ non activ nstextfield w...,objective-c cocoa nstextfield nstextview
19978,19997,necessari wsdl file creation webservic php que...,php web-services wsdl
19979,19998,java gc major gc vs minor gc latenc sensit app...,java garbage-collection


In [ ]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
multilabel_y = vectorizer.fit_transform(preprocessed_df['Tags'])

In [ ]:
def tags_to_consider(n):
    tag_i_sum = multilabel_y.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=multilabel_y[:,sorted_tags_i[:n]]
    #names=[vectorizer.get_feature_names() for i in multilabel_y[:,sorted_tags_i[:n]] ]
    return yn_multilabel
yx_multilabel = tags_to_consider(5500)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_df, yx_multilabel, test_size = 0.2,random_state = 42, )

In [ ]:
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=200, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))
x_train_multilabel = vectorizer.fit_transform(X_train['question'])
x_test_multilabel = vectorizer.transform(X_test['question'])

# Data loading

In [ ]:
rs = ShuffleSplit(n_splits=1, test_size=.3, random_state=0)
train_indices, test_indices = next(rs.split(preprocessed_df['question'].values))
train_indices, test_indices

(array([16922,  9484,  9116, ...,  9845, 10799,  2732]),
 array([18429, 16477,  4584, ...,  6109, 17395, 16140]))

In [ ]:
X_train, y_train = preprocessed_df.values[train_indices], yx_multilabel[train_indices]
X_test, y_test = preprocessed_df.values[test_indices], yx_multilabel[test_indices]
print(X_train.shape,X_test.shape)

(13986, 2) (5995, 2)


In [ ]:
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=200, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))
x_train_multilabel = vectorizer.fit_transform(X_train[:,0])
x_test_multilabel = vectorizer.transform(X_test[:,0])
print(x_train_multilabel.shape)

(13986, 200)


In [ ]:
x_train_multilabel.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.21506215, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

# Logistic Regression

In [ ]:
clf = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2'))
clf.fit(x_train_multilabel, y_train)
pickle.dump(clf,open('logistic.sav','wb'))

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 1771 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 1838 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 1934 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 2190 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 2201 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 2221 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 2225 is present in

In [ ]:
y_pred = clf.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))

Accuracy : 0.012176814011676397
Macro f1 score : 0.00024242067141159294
Micro f1 scoore : 0.03420038535645472


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


# SVM

In [ ]:
clf_svm = OneVsRestClassifier(SGDClassifier(alpha=0.001, penalty='l2'))
clf_svm.fit(X_train_multilabel, y_train)
pickle.dump(clf,open('svm.sav','wb'))

In [ ]:
y_pred_svm = clf.predict(X_test_multilabel)
print("Accuracy :",metrics.accuracy_score(y_test,y_pred_svm))
print("Macro f1 score :",metrics.f1_score(y_test, y_pred_svm, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred_svm, average = 'micro'))

# Naive Bayes